In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from mlxtend.preprocessing import TransactionEncoder
import a_funciones as fn

In [2]:
#### Conectar #######
conn=sql.connect('data/db_movies')
cur=conn.cursor()

## 1. Sistema de recomendación basado en popularidad 


In [3]:
#ver tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movie_final',),
 ('full_ratings',),
 ('reco',)]

In [4]:
full_ratings = pd.read_sql_query('select * from full_ratings', conn)
full_ratings.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [5]:
full_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34779 entries, 0 to 34778
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   34779 non-null  int64  
 1   movieId  34779 non-null  int64  
 2   rating   34779 non-null  float64
 3   title    34779 non-null  object 
 4   genres   34779 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 1.3+ MB


In [6]:
full_ratings['year'] = full_ratings['title'].str.extract(r'\((\d{4})\)').astype(int)
full_ratings['title'] = full_ratings['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()
full_ratings.head()

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,Grumpier Old Men,Comedy|Romance,1995
2,1,6,4.0,Heat,Action|Crime|Thriller,1995
3,1,47,5.0,Seven (a.k.a. Se7en),Mystery|Thriller,1995
4,1,50,5.0,"Usual Suspects, The",Crime|Mystery|Thriller,1995


In [7]:
full_ratings.info() #verificar que year sí esté como numeric 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34779 entries, 0 to 34778
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   34779 non-null  int64  
 1   movieId  34779 non-null  int64  
 2   rating   34779 non-null  float64
 3   title    34779 non-null  object 
 4   genres   34779 non-null  object 
 5   year     34779 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.6+ MB


In [8]:
# Conexión con SQLite 
conn = sql.connect(':memory:')
full_ratings.to_sql('full_ratings', conn, index=False, if_exists='replace')

34779

### a. Las películas más vistas, pero con calificación mayor a 4 (mejor calificadas)

In [9]:
## Las películas más vistas, pero con calificación mayor a 4
pd.read_sql("""select title, 
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(*) as seen_num
            from full_ratings
            group by title
            having avg_rat >= 4
            order by seen_num desc
            limit 10
            """, conn)



,title,avg_rat,seen_num
0,Forrest Gump,4.129412,255
1,Pulp Fiction,4.200422,237
2,"Shawshank Redemption, The",4.418502,227
3,"Matrix, The",4.132287,223
4,"Silence of the Lambs, The",4.158879,214
5,Star Wars: Episode IV - A New Hope,4.196078,204
6,Star Wars: Episode V - The Empire Strikes Back,4.208333,180
7,Fight Club,4.229050,179
8,American Beauty,4.077143,175
9,Star Wars: Episode VI - Return of the Jedi,4.101796,167


### b. Películas más vistas y mejores calificadas que sean para toda la familia

In [10]:
##Antes se debe desagregar los géneros para full_ratings
genres = full_ratings['genres'].str.split('|') # Divide la columna 'genres' en listas
te = TransactionEncoder() # Utiliza TransactionEncoder para convertirlo a variables dummy
genres_encoded = te.fit_transform(genres)
genres_full_ratings = pd.DataFrame(genres_encoded, columns=te.columns_) # Convierte el resultado en un dataframe con los nombres de las columnas correspondientes
full_ratings_dum = pd.concat([full_ratings, genres_full_ratings], axis=1) # Concatena el dataframe original con el dataframe de géneros codificados

# Elimina la columna
full_ratings_dum.drop(columns=['genres'], inplace=True) 

# Muestra los primeros registros del dataframe resultante
full_ratings_dum.head()

# Conexión con SQLite 
conn = sql.connect(':memory:')
full_ratings_dum.to_sql('full_ratings_dum', conn, index=False, if_exists='replace')

34779

In [11]:
## Las películas más vistas, con calificación mínima de 3.8 estrella, y que sean para toda la familia
pd.read_sql("""select title, 
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(*) as seen_num
            from full_ratings_dum
            where Children = 1 
            group by title
            having avg_rat >= 3.8
            order by seen_num desc
            limit 10
            """, conn)

,title,avg_rat,seen_num
0,Toy Story,3.881657,169
1,Shrek,3.851974,152
2,"Lion King, The",3.908163,147
3,Aladdin,3.842466,146
4,Finding Nemo,3.929752,121
5,"Monsters, Inc.",3.850877,114
6,"Incredibles, The",3.810811,111
7,Willy Wonka & the Chocolate Factory,3.943299,97
8,Toy Story 2,3.860465,86
9,"Wizard of Oz, The",3.886905,84


### c. Top mejores películas calificadas por año de estreno

In [12]:
# Conexión con SQLite 
conn = sql.connect(':memory:')
full_ratings.to_sql('full_ratings', conn, index=False, if_exists='replace')

34779

In [13]:

pd.read_sql("""select year, title, 
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(iif(rating = 0, Null, rating)) as rat_numb,
            count(*) as seen_num
            from full_ratings
            group by  year, title
            order by year desc, avg_rat desc limit 10
            """, conn)

,year,title,avg_rat,rat_numb,seen_num
0,2016,Deadpool,3.750000,42,42
1,2014,Guardians of the Galaxy,4.062500,48,48
2,2014,Interstellar,3.898148,54,54
3,2014,"Grand Budapest Hotel, The",3.776596,47,47
4,2013,"Wolf of Wall Street, The",3.848837,43,43
5,2012,Django Unchained,3.982759,58,58
6,2012,"Dark Knight Rises, The",3.928571,63,63
7,2012,"Avengers, The",3.889831,59,59
8,2012,The Hunger Games,3.480000,50,50
9,2010,Toy Story 3,4.063830,47,47


## 2.1 Sistema de recomendación basado en contenido KNN un solo producto seleccionado 

In [14]:
#### Conectar #######
conn=sql.connect('data/db_movies')
cur=conn.cursor()

In [15]:
df=pd.read_sql('select * from movie_final',conn)
df['movieId'] = df['movieId'].astype('object')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  436 non-null    object
 1   title    436 non-null    object
 2   genres   436 non-null    object
dtypes: object(3)
memory usage: 10.3+ KB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,6,Heat (1995),Action|Crime|Thriller
4,7,Sabrina (1995),Comedy|Romance


In [16]:
genres = df['genres'].str.split('|') # Divide la columna 'genres' en listas
te = TransactionEncoder() # Utiliza TransactionEncoder para convertirlo a variables dummy
genres_encoded = te.fit_transform(genres)
genres_df = pd.DataFrame(genres_encoded, columns=te.columns_) # Convierte el resultado en un dataframe con los nombres de las columnas correspondientes
df_dum = pd.concat([df, genres_df], axis=1) # Concatena el dataframe original con el dataframe de géneros codificados

# Elimina la columna original 'genres' y 'title'
df_dum.drop(columns=['genres'], inplace=True) 
df_dum.drop(columns=['title'], inplace=True)

# Muestra los primeros registros del dataframe resultante
df_dum.head()

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,2,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,6,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False
4,7,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False


In [17]:
model = neighbors.NearestNeighbors(n_neighbors=6, metric='cosine') 
model.fit(df_dum)
dist, idlist = model.kneighbors(df_dum)

In [18]:
distancias=pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila (pelicula)
id_list=pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde

In [19]:
def MovieRecommender(movie_name = list(df['title'].value_counts().index)):
    movie_list_name = []
    movie_id = df[df['title'] == movie_name].index[0]
    
    # Recomendaciones de películas similares
    for newid in idlist[movie_id]:
        movie_list_name.append(df.loc[newid].title)
    
    # Eliminar la película seleccionada de las recomendaciones
    if movie_name in movie_list_name:
        movie_list_name.remove(movie_name)
    
    return movie_list_name

# Interfaz interactiva
print(interact(MovieRecommender))


interactive(children=(Dropdown(description='movie_name', options=('Deadpool (2016)', 'Toy Story (1995)', 'Juma…

<function MovieRecommender at 0x000002573F45C180>


In [20]:
df_dum.to_csv('df_dum.csv', index=False)